In [1]:
#link google drive
!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials   
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [2]:
#get the csv file from google drive
link = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view'

# to get the id part of the file 
id = link.split("/")[-2] 

downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('SentimentTweets.csv')

In [3]:
#read from csv and store data
import pandas as pd 

df = pd.read_csv("SentimentTweets.csv",
                header = None, encoding = "ISO-8859-1",
                names=['val','target', 'id', 'date', 'flag', 'user', 'text'])

df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,val,target,id,date,flag,user,text
0,NaN,target,id,date,flag,user,text
1,680949.0,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
2,406741.0,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
3,1337108.0,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
4,1560887.0,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.


In [4]:
df.target.value_counts()

4         574622
0         574307
4          65987
0          65084
target         1
Name: target, dtype: int64

In [5]:
df1 = df.sample(n=10000)
df1.target.value_counts()

0    4527
4    4435
4     528
0     510
Name: target, dtype: int64

In [6]:
features = df1['text'].values
labels = df1['target'].values
features.shape

(10000,)

In [7]:
labels

array([0, '0', 4, ..., 4, 4, 4], dtype=object)

In [8]:
import re
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [9]:
#Use TfidfVectorizer to convert text into tokens/features
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

vectorizer = TfidfVectorizer (max_df=1.0, min_df=1, max_features=1000, stop_words='english')
processed_features = vectorizer.fit_transform(processed_features).toarray()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from sklearn.model_selection import train_test_split
labels=labels.astype('int')
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=4)

In [11]:
#Model Development
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
logreg = LogisticRegression(solver = 'lbfgs')

logreg.fit(X_train, y_train)
# make predictions on the testing set
y_pred = logreg.predict(X_test)

In [12]:
#Evaluation
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,y_pred))
print("Accuracy score:",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       984
           4       0.72      0.74      0.73      1016

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

Accuracy score: 0.7265
